This activity allows you to put lots of simulation skills to the test.

We are going to simulate a tokamak where you get to decide the materials.

The goal is to:

maximize the Tritium Breeding Ratio
minimize the heating on the center colum
maximize the heat in the blanket

In [71]:
import openmc


In [72]:
mat_conductor = openmc.Material(name='mat_conductor')
mat_conductor.add_element('Fe', 0.95, percent_type='wo')
mat_conductor.add_element('C', 0.05, percent_type='wo')
mat_conductor.set_density('g/cm3', 1)

mat_coolant = openmc.Material(name='mat_coolant')
mat_coolant.add_element('Fe', 0.95, percent_type='wo')
mat_coolant.set_density('g/cm3', 1)

mat_firstwall = openmc.Material(name='mat_firstwall')
mat_firstwall.add_element('Fe', 0.95, percent_type='wo')
mat_firstwall.set_density('g/cm3', 1)

mat_reflector = openmc.Material(name='mat_reflector')
mat_reflector.add_element('Fe', 0.95, percent_type='wo')
mat_reflector.set_density('g/cm3', 1)

mat_breeder = openmc.Material(name='mat_breeder')
mat_breeder.add_element('Pb', 84.2, percent_type='ao')
mat_breeder.add_element('Li', 15.8, percent_type='ao', enrichment=50.0, enrichment_target='Li6', enrichment_type='ao')
mat_breeder.set_density('g/cm3', 0.5)

my_materials = openmc.Materials([mat_conductor, mat_coolant, mat_firstwall, mat_breeder, mat_reflector])

In [73]:

# surfaces
central_column_surface_outer = openmc.ZCylinder(r=100)
central_column_surface_mid = openmc.ZCylinder(r=90)
central_column_surface_inner = openmc.ZCylinder(r=80)

inner_sphere_surface = openmc.Sphere(r=480)
middle_sphere_surface = openmc.Sphere(r=500) 
outer_sphere_surface = openmc.Sphere(r=600)
outer_outer_sphere_surface = openmc.Sphere(r=650)
edge_of_simulation_surface = openmc.Sphere(r=700, boundary_type='vacuum')

# regions
central_column_region = -central_column_surface_inner & -edge_of_simulation_surface
central_column_coolant_region = +central_column_surface_inner & -central_column_surface_mid & -edge_of_simulation_surface
central_column_fw_region = +central_column_surface_mid & -central_column_surface_outer & -edge_of_simulation_surface

inner_vessel_region = +central_column_surface_outer & -inner_sphere_surface

blanket_fw_region = -middle_sphere_surface & +inner_sphere_surface & +central_column_surface_outer
blanket_coolant_region = +middle_sphere_surface & -outer_sphere_surface & +central_column_surface_outer
blanket_breeder_region = +outer_sphere_surface & -outer_outer_sphere_surface & +central_column_surface_outer
blanket_reflector_region = +outer_outer_sphere_surface & -edge_of_simulation_surface & +central_column_surface_outer

# cells
central_column_cell = openmc.Cell(region=central_column_region, fill=mat_conductor)
central_column_coolant_cell = openmc.Cell(region=central_column_coolant_region, fill=mat_coolant)
central_column_fw_cell = openmc.Cell(region=central_column_fw_region, fill=mat_firstwall)
inner_vessel_cell = openmc.Cell(region=inner_vessel_region)
blanket_fw_cell = openmc.Cell(region=blanket_fw_region, fill=mat_firstwall)
blanket_coolant_cell = openmc.Cell(region=blanket_coolant_region, fill=mat_coolant)
blanket_breeder_cell  = openmc.Cell(region=blanket_breeder_region, fill=mat_breeder)
blanket_reflector_cell = openmc.Cell(region=blanket_reflector_region, fill=mat_reflector)

my_geometry = openmc.Geometry([
    central_column_cell,
    central_column_coolant_cell,
    central_column_fw_cell,
    inner_vessel_cell,
    blanket_fw_cell,
    blanket_coolant_cell,
    blanket_breeder_cell,
    blanket_reflector_cell,
])

# visualization
# plot = geometry.plot(basis='xz', legend=True, color_by='material', colors={mat_conductor:'red', mat_coolant:'blue', mat_firstwall:'grey', mat_breeder:'yellow', mat_reflector:'black'})
# plot.figure.savefig('xz-cell.png')

# plot = geometry.plot(basis='xy', legend=True, color_by='material', colors={mat_conductor:'red', mat_coolant:'blue', mat_firstwall:'grey', mat_breeder:'yellow', mat_reflector:'black'})
# plot.figure.savefig('xy-cell.png')


In [74]:
# initialises a new source object
my_source = openmc.IndependentSource()
# the distribution of radius is just a single value
radius = openmc.stats.Discrete([300], [1])
# the distribution of source z values is just a single value
z_values = openmc.stats.Discrete([0], [1])
# the distribution of source azimuthal angles values is a uniform distribution between 0 and 2 Pi
angle = openmc.stats.Uniform(a=0., b=2* 3.14159265359)
# this makes the ring source using the three distributions and a radius
my_source.space = openmc.stats.CylindricalIndependent(r=radius, phi=angle, z=z_values, origin=(0.0, 0.0, 0.0))
# sets the direction to isotropic
my_source.angle = openmc.stats.Isotropic()
# sets the energy distribution to a Muir distribution neutrons
my_source.energy = openmc.stats.muir(e0=14080000.0, m_rat=5.0, kt=20000.0)

my_settings = openmc.Settings()
my_settings.batches = 10
my_settings.inactive = 0
my_settings.particles = 500
my_settings.run_mode = 'fixed source'
my_settings.source = my_source

In [75]:
    # central_column_cell,
    # central_column_coolant_cell,
    # central_column_fw_cell,
    # inner_vessel_cell,
    # blanket_fw_cell,
    # blanket_coolant_cell,
    # blanket_breeder_cell,
    # blanket_reflector_cell,

cell_filter_breeder = openmc.CellFilter(blanket_breeder_cell)
tbr_tally = openmc.Tally(name='TBR')
tbr_tally.filters = [cell_filter_breeder]
tbr_tally.scores = ['(n,Xt)']

cell_filter_blanket_fw_coolant = openmc.CellFilter([
    blanket_breeder_cell,
    blanket_coolant_cell,
    blanket_fw_cell,
    central_column_coolant_cell,
    central_column_fw_cell,
])
blanket_heating_tally = openmc.Tally(name='heating')
blanket_heating_tally.filters = [cell_filter_blanket_fw_coolant]
blanket_heating_tally.scores = ['heating']

cell_filter_conductor = openmc.CellFilter(blanket_breeder_cell)
conductor_damage_tally = openmc.Tally(name='conductor_damage')
conductor_damage_tally.filters = [cell_filter_conductor]
conductor_damage_tally.scores = ['444']

my_tallies = openmc.Tallies([tbr_tally, blanket_heating_tally, conductor_damage_tally])

TypeError: Unable to set "tally filters" to "Cell
	ID             =	137
	Name           =	
	Fill           =	Material 66
	Region         =	(-139 -144)
	Rotation       =	None
	Temperature    =	None
	Translation    =	None
	Volume         =	None
" which is not one of the following types: "Filter, CrossFilter, AggregateFilter"

In [ ]:
model = openmc.model.Model(my_geometry, my_materials, my_settings, my_tallies)

# removes the old output files
!rm *.h5

sp_filename = model.run()



In [ ]:
sp = openmc.StatePoint(sp_filename)

tbr_tally = sp.get_tally(name='TBR')

print(f'TBR={tbr_tally.mean.flatten()[0]} with standard deviation of {tbr_tally.std_dev.flatten()[0]}')

heating_tally = sp.get_tally(name='heating')
print(f'Heating={heating_tally.mean.flatten()[0]/1e6}MeV per source neutron with standard deviation of {heating_tally.std_dev.flatten()[0]}')

damage_tally = sp.get_tally(name='conductor_damage')
print(f'damage={damage_tally.mean.flatten()[0]} with standard deviation of {damage_tally.std_dev.flatten()[0]}')

